In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nadabl","key":"8e8ff7b85fb1a8cb444502949cc560ec"}'}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 kaggle.json ~/.kaggle/

In [ ]:
! kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store

100% 4.29G/4.29G [01:25<00:00, 51.6MB/s]
100% 4.29G/4.29G [01:25<00:00, 53.9MB/s]


In [ ]:
! unzip ecommerce-behavior-data-from-multi-category-store

Archive:  ecommerce-behavior-data-from-multi-category-store.zip
  inflating: 2019-Nov.csv            
  inflating: 2019-Oct.csv            


In [ ]:
#pandas takes the most time (2 mins)
import pandas as pd 
dt=pd.read_csv("/content/2019-Oct.csv")

In [ ]:
pip install dask   

In [ ]:
pip install 'fsspec>=0.3.3'

In [ ]:
# almost 2secs for Dask
import dask.dataframe as dd
dt = dd.read_csv("/content/2019-Oct.csv")

In [ ]:
pip install modin

In [ ]:
pip install modin[dask]

In [ ]:
import modin.pandas as pd
#almost 2secs too
dt = pd.read_csv("/content/2019-Oct.csv")

In [ ]:
dt.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [ ]:
dt.columns

Index(['event_time', 'event_type', 'product_id', 'category_id',
       'category_code', 'brand', 'price', 'user_id', 'user_session'],
      dtype='object')

In [ ]:
dt.shape

(42448764, 9)

In [ ]:
dt.size

382038876

In [ ]:
#file size in bytes(5.27GB)
import os
os.path.getsize("/content/2019-Oct.csv")

5668612855

In [ ]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)

def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing testutility.py


In [ ]:
%%writefile file.yaml
file_type: csv
dataset_name: dt
file_name: 2019-Oct
table_name: edsurv
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    -event_time
    -event_type
    -product_id
    -category_id
    -category_code
    -brand
    -price
    -user_id
    -user_session


Writing file.yaml


In [ ]:
import testutility as util
config_data = util.read_config_file("file.yaml")


In [ ]:
config_data['file_name']

'2019-Oct'

In [ ]:
#inspecting data of config file
config_data

{'columns': '-event_time -event_type -product_id -category_id -category_code -brand -price -user_id -user_session',
 'dataset_name': 'dt',
 'file_name': '2019-Oct',
 'file_type': 'csv',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'table_name': 'edsurv'}

In [ ]:
# read the file using config file
file_type = config_data['file_type']
source_file = "/content/" + config_data['file_name'] + f'.{file_type}'
#print("",source_file)
df = pd.read_csv(source_file)
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [ ]:
#validate the header of the file
util.col_header_val(df,config_data)

column name and column length validation failed
Following File columns are not in the YAML file ['user_id', 'price', 'category_id', 'product_id', 'event_time', 'user_session', 'event_type', 'brand', 'category_code']
Following YAML columns are not in the file uploaded ['y', 'a', 'd', '_', 'c', 's', 'b', 'e', 'p', 'r', 'v', 't', 'm', ' ', 'o', 'u', 'n', 'g', 'i', '-']


0

In [ ]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['event_time', 'event_type', 'product_id', 'category_id',
       'category_code', 'brand', 'price', 'user_id', 'user_session'],
      dtype='object')
columns of YAML are: -event_time -event_type -product_id -category_id -category_code -brand -price -user_id -user_session


In [ ]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine

column name and column length validation failed
Following File columns are not in the YAML file ['user_id', 'price', 'category_id', 'product_id', 'event_time', 'user_session', 'event_type', 'brand', 'category_code']
Following YAML columns are not in the file uploaded ['y', 'a', 'd', '_', 'c', 's', 'b', 'e', 'p', 'r', 'v', 't', 'm', ' ', 'o', 'u', 'n', 'g', 'i', '-']
validation failed
